In [ ]:
%python
%pip install faker

In [ ]:
%python
dbutils.library.restartPython() 

In [ ]:
%python
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from faker import Faker
import random

spark = SparkSession.builder.getOrCreate()
fake = Faker("pt_BR")
Faker.seed(42)
random.seed(42)

In [ ]:
%python
from pyspark.sql.types import StructType, StructField, StringType

schema_clientes_outros = StructType([
    StructField("nome", StringType(), True),
    StructField("cpf", StringType(), True),
    StructField("telefone", StringType(), True),
    StructField("email", StringType(), True),
    StructField("endereco", StringType(), True),
    StructField("id_cliente", StringType(), True),
    StructField("id_banco", StringType(), True),
    StructField("sigla_estado", StringType(), True),
    StructField("cidade", StringType(), True),
    StructField("data_nascimento", StringType(), True),
    StructField("uuid", StringType(), True),
    StructField("chave_pix", StringType(), True),
])

df_clientes_outros = spark.createDataFrame(
    clientes_outros_data,
    schema_clientes_outros
)
df_clientes_outros.createOrReplaceTempView("clientes_outros")
df_bancos.write.mode("overwrite").saveAsTable("corebank_catalog.bronze_corebank.clientes_outros")
display(df_clientes_outros)

In [ ]:
%python
qtde_clientes_outros = 300

clientes_outros_data = []
for i in range(1, qtde_clientes_outros + 1):
    sigla_estado, nome_estado = fake.estado()
    cidade = fake.city()
    id_banco = random.choice(bancos_ids)
    nome = fake.name()
    cpf = fake.cpf()
    telefone = fake.phone_number()
    email = fake.email()
    endereco = fake.address().replace("\n", ", ")
    id_cliente = f"X{i:06d}"
    data_nascimento = fake.date_of_birth().isoformat()
    uuid = fake.uuid4()
    chave_pix = random.choice([cpf, telefone, email])  # Pix key is randomly one of these

    clientes_outros_data.append((
        nome,
        cpf,
        telefone,
        email,
        endereco,
        id_cliente,
        id_banco,
        sigla_estado,
        cidade,
        data_nascimento,
        uuid,
        chave_pix
    ))

df_clientes_outros = spark.createDataFrame(clientes_outros_data, schema_clientes_outros)
df_clientes_outros.createOrReplaceTempView("clientes_outros")